In [41]:
from sympy import symbols, diff, Eq, solve, lambdify

# Define variables
y, lambda_symbol = symbols('y lambda')

# Define b(y)
b_y = 1 / (100 + y) ** 2

# Define Profit function P(y)
P_y = y * b_y

# Find derivative of Profit function and solve for y
P_prime_y = diff(P_y, y)
critical_points = solve(P_prime_y, y)

# Check the critical points found to determine the value of y that maximizes profit
y_max_profit = critical_points[0] if len(critical_points) > 0 else 0

# Define the Lagrangian L = P(y) + λ(ȳ - y)
y_bar = symbols('y_bar')  # ȳ
Lagrangian = P_y + lambda_symbol * (y_bar - y)

# Solve the Lagrangian for given ȳ values
y_bars = [75, 150]
lagrange_multipliers = []

for y_value in y_bars:
    # Take the derivative of Lagrangian with respect to y and λ, and solve
    dL_dy = diff(Lagrangian.subs(y_bar, y_value), y)
    dL_dlambda = diff(Lagrangian.subs(y_bar, y_value), lambda_symbol)
    solutions = solve((dL_dy, dL_dlambda), (y, lambda_symbol))

    print(solutions)    
    
    lagrange_multipliers.append({
        'y_bar': y_value,
        'y': solutions[0][0],
        'lambda': solutions[0][1]
    })

# Print Results
print(f"To maximize profit, buy and sell {y_max_profit.evalf()} units.")
for lagrange_multiplier in lagrange_multipliers:
    print(f"For y_bar = {lagrange_multiplier['y_bar']}, y = {lagrange_multiplier['y'].evalf()}, λ = {lagrange_multiplier['lambda'].evalf()}")


[(75, 1/214375)]
[(150, -1/312500)]
To maximize profit, buy and sell 100.000000000000 units.
For y_bar = 75, y = 75.0000000000000, λ = 0.00000466472303206997
For y_bar = 150, y = 150.000000000000, λ = -0.00000320000000000000


In [42]:
from amplpy import AMPL, Environment

ampl = AMPL()  # Replace with your AMPL installation path

# Define the variables with their respective bounds
ampl.eval('var x1 >= 0;')
ampl.eval('var x2 >= 0;')
ampl.eval('var x3 >= 0;')

# Define the objective function
ampl.eval('maximize Profit: 10*x1/(1 + x1) + sqrt(x2) + 10*(1 - exp(-x3));')

# Define the constraints
ampl.eval('s.t. Budget: x1 + x2 + x3 <= 50;')

# Choose a solver
ampl.setOption('solver', 'gurobi')  # Replace with your solver
ampl.eval("option gurobi_options 'nonconvex=2';")

# Solve the problem
ampl.solve()

# Get the results
x1 = ampl.getVariable('x1').get().value()
x2 = ampl.getVariable('x2').get().value()
x3 = ampl.getVariable('x3').get().value()

lagrange_multiplier = ampl.getConstraint('Budget').get_values().to_pandas()


# Display the results
print(f'Optimal investments are x1 = {x1}, x2 = {x2}, x3 = {x3}')


Gurobi 10.0.3:   qp:nonconvex = 2
Gurobi 10.0.3: optimal solution; objective 24.94163745
26932 simplex iterations
2464 branching nodes
absmipgap=0.00179202, relmipgap=7.18487e-05
Optimal investments are x1 = 9.897231162050723, x2 = 35.35276883794928, x3 = 4.75
Lagrange multiplier (shadow price) of the Budget constraint is    Budget.dual
0            0


In [43]:
import math

def f(lamb):
    return math.sqrt(10 / lamb) - 1 + 1 / (4 * lamb ** 2) + math.log(10 / lamb) - 50

# Derivative of the function
def df(lamb):
    return -5 / (2 * lamb ** 1.5) - 1 / (2 * lamb ** 3) - 1 / lamb

# Initial guess
lamb = 0.1 
max_iter = 10000000
tol = 1e-12

for i in range(max_iter):
    lamb_new = lamb - f(lamb) / df(lamb)  # Newton-Raphson update
    
    if abs(lamb_new - lamb) < tol:
        break
    
    lamb = lamb_new
else:
    print("The method did not converge")


x1 = math.sqrt(10/lamb_new)-1
x2 = 1/(4*lamb_new**2)
x3=math.log(10/lamb_new)

print("Lambda: ", lamb_new)
print(f'Optimal investments are x1 = {x1}, x2 = {x2}, x3 = {x3}')


Lambda:  0.08413203492281647
Optimal investments are x1 = 9.90232949385604, x2 = 35.31971754327805, x3 = 4.777952962867916


In [44]:
import math

def f(lamb):
    return math.sqrt(10 / lamb) - 1 + 1 / (4 * lamb ** 2) + math.log(10 / lamb) - 50

# Define the derivative of the function
def df(lamb):
    return - (-5 / (2 * lamb ** 1.5)) - 1 / (2 * lamb ** 3) - 1 / lamb

# Initial guess
lamb = 0.1  # This is just a guess, it might need to be adjusted

# Iteration count
max_iter = 10000000

# Tolerance
tol = 1e-12

for i in range(max_iter):
    lamb_new = lamb - f(lamb) / df(lamb)  # Newton-Raphson update
    
    # Check for convergence
    if abs(lamb_new - lamb) < tol:
        break
    
    lamb = lamb_new
else:
    print("The method did not converge")


x1 = math.sqrt(10/lamb_new)-1
x2 = 1/(4*lamb_new**2)
x3=math.log(10/lamb_new)

print("Lambda: ", lamb_new)
print(f'Optimal investments are x1 = {x1}, x2 = {x2}, x3 = {x3}')


Lambda:  0.08413203492286606
Optimal investments are x1 = 9.902329493852827, x2 = 35.31971754323642, x3 = 4.777952962867326


In [45]:
from sympy import symbols, Eq, solve, log

# Define variables
n = symbols('n', integer=True)  # number of assets
x = symbols('x:%d' % n)  # fractions of wealth invested in assets
x0 = symbols('x0')  # fraction of wealth not invested
p = symbols('p:%d' % n)  # probabilities of winning
b = symbols('b:%d' % n)  # returns
λ = symbols('λ')  # Lagrange multiplier for the sum constraint
μ = symbols('μ:%d' % (n + 1))  # Lagrange multipliers for the ≥ 0 constraints

# Define the utility function U(x)
U = sum(p[i] * x[i] * log(1 + b[i]) for i in range(n)) + x0 * log(1)

# Define the Lagrangian L
L = U + λ * (sum(x) + x0 - 1) - sum(μ[i] * x[i] for i in range(n)) - μ[n] * x0

# Define the KKT conditions
KKT_conditions = [Eq(L.diff(x[i]), 0) for i in range(n)] + [Eq(L.diff(x0), 0)]

# Solve the KKT conditions
solution = solve(KKT_conditions, dict=True)

# Analyze the solution to deduce whether x0* > 0
# You would typically look at the values of the Lagrange multipliers and the dual variables to draw conclusions about x0*.


TypeError: %d format: a real number is required, not Symbol